In [98]:
import requests
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
from d3blocks import d3blocks
from tqdm import tqdm

In [99]:
searchurl = "https://ilthermo.boulder.nist.gov/ILT2/ilsearch" 

In [100]:
searchdata = {
    "prp":"jVUM"
    }

In [101]:
r = requests.get(url=searchurl, params=searchdata) #search in search api with target property 

In [65]:
r.json()['res'][0]

['tSjaP',
 'Alcalde et al. (2018)',
 'Viscosity',
 'Liquid',
 'ABchlW',
 'AAIsuc',
 None,
 '1000',
 '1-butyl-3-methylimidazolium bis(trifluoromethylsulfonyl)imide',
 'ethanol']

In [102]:
i = [] #create list for setid
for sets in r.json()['res']: 
    if 'Liquid' in sets[3]: #check if phase is liquid
        i.append(sets[0]) #get setid from search api results & append to list
        #print(i)

In [103]:
seturl = "https://ilthermo.boulder.nist.gov/ILT2/ilset"

In [118]:
i = ['eAgTr']

In [119]:
d = {}

In [105]:
def molality(inpt,keys,mass):
    mlt = []
    i = 0
    for kcount, key in enumerate(keys):
        if not len(key) > 1:
            continue
        if float(inpt[key[1]][0]) == 0:
            mole = 0
        else:
            mole = 1 / (1 / (float(inpt[key[1]][0]) * 0.001 * float(mass)) + 1)
            
        i += mole
        
        mlt.insert(kcount,mole)
    
    for kcount, key in enumerate(keys):
        if not len(key) == 1:
            continue
        mole = 1 - i
        
        mlt.insert(kcount,mole)
        
    return(mlt)

In [106]:
def molepermass(inpt,keys):
    moles = []
    mpms = []
    i = 0
    totalmole = 0
    for kcount, key in enumerate(keys):
        if len(key) > 1:
            mole = float(inpt[key[1]][0])
            i += mole * float(key[0]) #mole * g/mole = g
            
            moles.insert(kcount,mole)
    
    for kcount, key in enumerate(keys):
        if len(key) == 1:
            mole = 1000 - i 
            
            moles.insert(kcount,mole)
    
    for mole in moles:
        totalmole += mole
        
    for mole in moles:
        if mole == 0:
            mpm = 0
        else:
            mpm = mole / totalmole
        mpms.append(mpm)
        
    return(mpms)

In [107]:
def weightfrac(inpt,keys):
    moles = []
    wfs = []
    i = 0
    totalmole = 0
    for kcount, key in enumerate(keys):
        if len(key) > 1:
            mole = float(inpt[key[1]][0]) / float(key[0])
            i += float(inpt[key[1]][0])
            
            moles.insert(kcount,mole)
    
    for kcount, key in enumerate(keys):
        if len(key) == 1:
            mole = 1 - i / float(key[0])
            
            moles.insert(kcount,mole)
    
    for mole in moles:
        totalmole += mole
        
    for mole in moles:
        if mole == 0:
            wf = 0
        else:
            wf = mole / totalmole
        wfs.append(wf)
        
    return(wfs)

In [108]:
def molefrac(inpt,keys):
    mfs = []
    i = 0
    for kcount, key in enumerate(keys):
        if len(key) > 1:
            mf = float(inpt[key[1]][0])
            i += float(inpt[key[1]][0])
            
            mfs.insert(kcount,mf)
    
    for kcount, key in enumerate(keys):
        if len(key) == 1:
            mf = 1 - i
            
            mfs.insert(kcount,mf)
        
    return(mfs)

In [109]:
def moleratio(inpt,keys):
    moles = []
    mrs = []
    i = 0
    totalmole = 1
    for kcount, key in enumerate(keys):
        if len(key) > 1:
            mole = float(inpt[key[1]][0])
            i += float(inpt[key[1]][0])
            
            moles.insert(kcount,mole)
    
    for kcount, key in enumerate(keys):
        if len(key) == 1:
            mole = 1 - i
            
            moles.insert(kcount,mole)
    
    for mole in moles:
        totalmole += mole
        
    for mole in moles:
        mr = mole / totalmole
        mrs.append(mr)
        
    return(mrs)

In [120]:
#define x y headings
xheading = 'Pressure, kPa'
yheading = 'Temperature, K'
zheading = 'Viscosity, Pa&#8226;s'
failed = []

plt.figure(figsize=(16, 9))
#search in set api with setid
for ids in tqdm(i):
    setdata = {
    "set":ids
    }
    #print(setdata)
    r = requests.get(url=seturl, params=setdata) 
    url = (r.url)
    
    #r.json()['components']
    
    title = r.json()['ref']['title']
    cite = r.json()['ref']['full']
    
    #define componenets
    
    solutes = []
    solvents = []
    for ccount, component in enumerate(r.json()['components']):
        if r.json()['solvent'] is None:
            solutes.append([r.json()['components'][ccount]['name'], r.json()['components'][ccount]['mw']])
        elif r.json()['components'][ccount]['name'] in r.json()['solvent']:
            solvents.append([r.json()['components'][ccount]['name'], r.json()['components'][ccount]['mw']])
        else:
            solutes.append([r.json()['components'][ccount]['name'], r.json()['components'][ccount]['mw']])
    
    #define ionic liquid
    il = ", ".join([solute[0] for solute in solutes]) + ", " + ", ".join([solvent[0] for solvent in solvents])

        
    #print(solvents)
    #print(il)
    
    slkeys = []
    for slcount, sl in enumerate(solutes):
        slkeys.append([solutes[slcount][1]])
        
    svkeys = []
    for svcount, sv in enumerate(solvents):
        svkeys.append([solvents[svcount][1]])
    
    failcheck = []
    
    #filter out necessary data through heading 
    dhead = (r.json()['dhead'])
    data = (r.json()['data'])
    soluteX = []
    solventX = []
    xvalue = []
    yvalue = []
    zvalue = []
    datakeys = []
    for hcount, heads in enumerate(dhead):
        if heads[0] == xheading:
            datakeys.insert(0,hcount)
            
        elif heads[0] == yheading:
            datakeys.insert(1,hcount)
            
        elif heads[0] == zheading:
            datakeys.insert(2,hcount)
            
        else:
            for slkcount, slkey in enumerate(slkeys):
                if solutes[slkcount][0] in heads[0]:
                    slkeys[slkcount].append(hcount)
                    break
            #sample key: [[slmass,location],[slmass]]
                
            for svkcount, svkey in enumerate(svkeys):
                if solvents[svkcount][0] in heads[0]:
                    svkeys[svkcount].append(hcount)
                    break
            #sample key: [[svmass,location],[svmass]]

              
    if not len(datakeys) == 3:
        failed.append(ids)
        continue
        
        #print(datakeys)
        print(slkeys)
        print(svkeys)
    
    for values in data:
        xvalue.append(values[datakeys[0]])
        yvalue.append(values[datakeys[1]])
        zvalue.append(values[datakeys[2]])
        
    svmass = []
    if len(solvents) == 2:
        A = []
        B = []
        for svkcount, svkey in enumerate(svkeys): #check solvent unit
            if not len(svkeys[svkcount]) > 1: #check if data available(continue if not)
                continue
            if "MolaLity" in dhead[svkeys[svkcount][1]][0]:
                if solvents[0][0] in dhead[svkeys[svkcount][1]][0]:
                    for values in data:
                        A.append(molality(values,svkeys,solvents[1][1])[0]) #1st
                        B.append(molality(values,svkeys,solvents[1][1])[1]) #2nd
                        svmas = float(solvents[0][1]) * molality(values,svkeys,solvents[1][1])[0] + float(solvents[1][1]) * molality(values,svkeys,solvents[1][1])[1]
                        svmass.append(svmas)
                elif solvents[1][0] in dhead[svkeys[svkcount][1]][0]:
                    for values in data:
                        A.append(molality(values,svkeys,solvents[0][1])[0]) #1st
                        B.append(molality(values,svkeys,solvents[0][1])[1]) #2nd    
                        svmas = float(solvents[0][1]) * molality(values,svkeys,solvents[0][1])[0] + float(solvents[1][1]) * molality(values,svkeys,solvents[0][1])[1]
                        svmass.append(svmas)
                break #break forloop once unit is determined
            
            elif "Weight fraction" in dhead[svkeys[svkcount][1]][0]:
                for values in data:
                    A.append(weightfrac(values,svkeys)[0])
                    B.append(weightfrac(values,svkeys)[1])
                    svmas = float(solvents[0][1]) * weightfrac(values,svkeys)[0] + float(solvents[1][1]) * weightfrac(values,svkeys)[1]
                    svmass.append(svmas)
                break
                
            elif "Mole fraction" in dhead[svkeys[svkcount][1]][0]:
                for values in data:
                    A.append(molefrac(values,svkeys)[0])
                    B.append(molefrac(values,svkeys)[1])
                    svmas = float(solvents[0][1]) * molefrac(values,svkeys)[0] + float(solvents[1][1]) * molefrac(values,svkeys)[1]
                    svmass.append(svmas)
                break
                
            elif "Moles per mass of solution:" in dhead[svkeys[svkcount][1]][0]:
                for values in data:
                    A.append(molepermass(values,svkeys)[0])
                    B.append(molepermass(values,svkeys)[1])
                    svmas = float(solvents[0][1]) * molepermass(values,svkeys)[0] + float(solvents[1][1]) * molepermass(values,svkeys)[1]
                    svmass.append(svmas)
                break
                
            elif "Mole ratio" in dhead[svkeys[svkcount][1]][0]:
                for values in data:
                    A.append(moleratio(values,svkeys)[0])
                    B.append(moleratio(values,svkeys)[1])
                    svmas = float(solvents[0][1]) * moleratio(values,svkeys)[0] + float(solvents[1][1]) * moleratio(values,svkeys)[1]
                    svmass.append(svmas)
                break
            else:
                failcheck.append(ids)
                break

    elif len(solvents) == 1:
        for values in data:
            svmass.append(float(solvents[0][1]))
            
    elif len(solvents) == 0:
        for values in data:
            svmass.append(0)
    else:
        failcheck.append(ids)
        break
        
        
    if not len(failcheck) == 0:
        failed.append(ids)
        continue
        
    if len(solutes) == 1 and len(solvents) == 0:
        for values in data:
            soluteX.append(1)
    else:
        for slkcount, slkey in enumerate(slkeys):
            if not len(slkeys[slkcount]) > 1:
                continue
            if "MolaLity" in dhead[slkeys[slkcount][1]][0]:
                for slkcount, slkey in enumerate(slkeys):
                    slX = []
                    for dcount, values in enumerate(data):
                        slX.append(molality(values,slkeys,svmass[dcount])[slkcount])
                    soluteX.append(slX)
                break

            elif "Weight fraction" in dhead[slkeys[slkcount][1]][0]:
                for slkcount, slkey in enumerate(slkeys):
                    slX = []
                    for dcount, values in enumerate(data):
                        slX.append(weightfrac(values,slkeys)[slkcount])
                    soluteX.append(slX)
                break

            elif "Mole fraction" in dhead[slkeys[slkcount][1]][0]:
                for slkcount, slkey in enumerate(slkeys):
                    slX = []
                    for dcount, values in enumerate(data):
                        slX.append(molefrac(values,slkeys)[slkcount])
                    soluteX.append(slX)
                break

            elif "Moles per mass of solution:" in dhead[slkeys[slkcount][1]][0]:
                for slkcount, slkey in enumerate(slkeys):
                    slX = []
                    for dcount, values in enumerate(data):
                        slX.append(molepermass(values,slkeys)[slkcount])
                    soluteX.append(slX)
                break

            elif "Mole ratio" in dhead[slkeys[slkcount][1]][0]:
                for slkcount, slkey in enumerate(slkeys):
                    slX = []
                    for dcount, values in enumerate(data):
                        slX.append(moleratio(values,slkeys)[slkcount])
                    soluteX.append(slX)
                break
            else:
                failcheck.append(ids)
                break
    
    if len(soluteX) == 0:
        failed.append(ids)
        continue
    
    
    if not len(failcheck) == 0:
        failed.append(ids)
        continue

    if len(solvents) == 2:
        AX = []
        BX = []
        for dcount, values in enumerate(data):
            svratio = 1
            for slkcount, slkey in enumerate(slkeys):
                svratio -= soluteX[slkcount][dcount]
            aX = svratio * A[dcount]
            bX = svratio * B[dcount]
            AX.append(aX)
            BX.append(bX)
        solventX.append(AX)
        solventX.append(BX)
        
    elif len(solvents) == 1:
        AX = []
        for dcount, values in enumerate(data):
            svratio = 1
            for slkcount, slkey in enumerate(slkeys):
                svratio -= soluteX[slkcount][dcount]
            aX = svratio
            AX.append(aX)
        solventX.append(AX)   
            

    #print(A)
    #print(B)
    #print(soluteX)                   
                        
    #print(len(data))                    
    #print(len(solventX[0]))
    #print(len(solventX[1]))
    #print(len(soluteX[0]))
    #print(solventX)
    #print(soluteX)
    #print(title)
    #print(cite)
    
                
    #print(dhead)
    #convert list in data into strings + separate error(if any)
    xdata = []
    xerr = []
    ydata = []
    yerr = []
    zdata = []
    zerr = []
    
    if len(xvalue[0]) > 1:
        for datas in xvalue:
            xdata.append(datas[0])
            xerr.append(datas[1])
    else:
        for datas in xvalue:
            xdata.append(datas[0])
            
    if len(yvalue[0]) > 1:
        for datas in yvalue:
            ydata.append(datas[0])
            yerr.append(datas[1])
    else:
        for datas in yvalue:
            ydata.append(datas[0])
            
    if not len(zvalue) == 0: 
        if len(zvalue[0]) > 1:
            for datas in zvalue:
                zdata.append(datas[0])
                zerr.append(datas[1])
        else:
            for datas in zvalue:
                zdata.append(datas[0])
        
    if il not in d.keys():
        d[il] = {
            title : {"citation": cite,
                "solutes": solutes,
                "solvents": solvents,
                "soluteX": soluteX, 
                "solventX": solventX, 
                "xd": xdata, 
                "xe": xerr, 
                "yd": ydata, 
                "ye": yerr,
                "zd": zdata, 
                "ze": zerr
                    }
            }
    else:
        d[il][title] = {
            "citation": cite,
            "solutes": solutes,
            "solvents": solvents,
            "soluteX": soluteX, 
            "solventX": solventX, 
            "xd": xdata, 
            "xe": xerr, 
            "yd": ydata, 
            "ye": yerr,
            "zd": zdata, 
            "ze": zerr
            }

100%|█████████████████████████████████████████████| 1/1 [00:01<00:00,  1.88s/it]


<Figure size 1152x648 with 0 Axes>

In [116]:
with open('Viscosity.json','w') as f:
    json.dump(d,f)